<center> <img src = "https://raw.githubusercontent.com/denis-marchenkov-sf/assets/aed651ad29de045160567af37cd221ec604c6baa/hh%20label.jpg" alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [299]:
import pandas as pd
import psycopg2
import plotly.express as px
import requests
from bs4 import BeautifulSoup

# отключим назойливые предупреждения о pandas коннекторе
import warnings
warnings.filterwarnings('ignore', message = '.*SQLAlchemy connectable')

In [300]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
# REDACTED :)

In [301]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Количество вакансий (таблица vacancies).

In [302]:
# текст запроса
query_3_1 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies

'''

In [303]:
# результат запроса
df_vacancy_total = pd.read_sql_query(query_3_1, connection)

print('Количество вакансий:')
display(df_vacancy_total)


Количество вакансий:


,count
0,49197


2. Количество работодателей (таблица employers). 

In [304]:
# текст запроса
query_3_2 = f'''

SELECT
    COUNT(*)
FROM
    public.employers

'''

In [305]:
# результат запроса
df_employer_total = pd.read_sql_query(query_3_2, connection)

print('Количество работодателей:')
display(df_employer_total)

Количество работодателей:


,count
0,23501


3. Количество регионов (таблица areas).

In [306]:
# текст запроса
query_3_3 = f'''

SELECT
    COUNT(*)
FROM
    public.areas

'''

In [307]:
# результат запроса
df_region_total = pd.read_sql_query(query_3_3, connection)

print('Количество регионов:')
display(df_region_total)

Количество регионов:


,count
0,1362


4. Количество сфер деятельности (таблица industries).

In [308]:
# текст запроса
query_3_4 = f'''

SELECT
    COUNT(*)
FROM
    public.industries

'''

In [309]:
# результат запроса
df_industry_total = pd.read_sql_query(query_3_4, connection)

print('Количество сфер деятельности:')
display(df_industry_total)


Количество сфер деятельности:


,count
0,294


# Выводы по предварительному анализу данных
Всего в базе представлено 49197 вакансий по 1362 регионам в 294 сферах деятельности от 23501 работодателя.

***

# Юнит 4. Детальный анализ вакансий

1. Количество вакансий в каждом регионе.

In [310]:
# текст запроса
query_4_1 = f'''

SELECT
    area.name           AS area,
    COUNT(vacancy.id)   AS cnt
FROM
    public.vacancies AS vacancy
JOIN
    public.areas AS area ON vacancy.area_id = area.id
GROUP BY
    area.id
ORDER BY cnt DESC

'''

In [311]:
# результат запроса
df_vacancy_region = pd.read_sql_query(query_4_1, connection)

print('Количество вакансий по регионам в порядке убывания:')
display(df_vacancy_region)

Количество вакансий по регионам в порядке убывания:


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


2. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [312]:
# текст запроса
query_4_2 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies
WHERE
    salary_from IS NOT NULL
    OR
    salary_to IS NOT NULL

'''

In [313]:
# результат запроса
df_vacancy_not_null = pd.read_sql_query(query_4_2, connection)

print('Количество вакансий с указанной зарплатой:')
display(df_vacancy_not_null)

Количество вакансий с указанной зарплатой:


,count
0,24073


3. Cредние значения для нижней и верхней границы зарплатной вилки округлённые до целого.

In [314]:
# текст запроса
query_4_3 = f'''

SELECT
    ROUND( AVG(salary_from) )   AS avg_from,
    ROUND( AVG(salary_to) )     AS avg_to
FROM
    public.vacancies

'''

In [315]:
# результат запроса
df_avg_salary = pd.read_sql_query(query_4_3, connection)

print('Cредние значения для нижней и верхней границы зарплатной вилки:')
display(df_avg_salary)

Cредние значения для нижней и верхней границы зарплатной вилки:


,avg_from,avg_to
0,71065.0,110537.0


4. Количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.


In [316]:
# текст запроса
query_4_4 = f'''

SELECT
    schedule,
    employment,
    COUNT(id) AS cnt
FROM
    public.vacancies
GROUP BY
    schedule, employment
ORDER BY cnt DESC

'''

In [317]:
# результат запроса
df_schedule = pd.read_sql_query(query_4_4, connection)

print('Количество вакансий по типам рабочего графика и трудоустройства:')
display(df_schedule)

Количество вакансий по типам рабочего графика и трудоустройства:


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Значения поля "Требуемый опыт работы" (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [318]:
# текст запроса
query_4_5 = f'''

SELECT
    experience,
    COUNT(id) AS cnt
FROM
    public.vacancies
WHERE
    experience IS NOT NULL
GROUP BY
    experience
ORDER BY cnt ASC

'''

In [319]:
# результат запроса
df_experience = pd.read_sql_query(query_4_5, connection)

print('Требуемый опыт работы:')
display(df_experience)

Требуемый опыт работы:


,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


# Выводы по детальному анализу вакансий

Москва, Санкт-Петербург, Минск, Новосибирск и Алматы входят в пятёрку лидеров по количеству вакансий.

У половины вакансий не указана заработная плата.

Средние показатели заработной платы лежат в диапазоне от 70 до 110 тысяч.

Наиболее востребованы сотрудники на полную занятость и полный рабочий день.

Свыше половины вакансий требуют наличие опыта работы от 1 года до 3 лет.

<div>
    <img src="images/vacancy_by_region.png">
</div>
<br/>
<div>
    <img src="images/salary_specified.png">
</div>
<br/>
<div>
    <img src="images/schedule_employment.png">
</div>
<br/>
<div>
    <img src="images/experience.png">
</div>

***

In [320]:
# построение графиков детального анализа вакансий

# города-лидеры по вакансиям
bar_vacancy = px.bar(
    df_vacancy_region.head(5),
    x = 'area',
    y = 'cnt',
    color = 'area',
    color_discrete_sequence = ['orange', 'yellow', 'green', 'blue', 'teal'],
    labels = { 'cnt':'Количество вакансий', 'area':'Город' },
    title = 'Распределение количества вакансий в первой пятёрке городов'
)

bar_vacancy.update_layout(
    title_x = 0.5
)

bar_vacancy.write_image("images/vacancy_by_region.png")

bar_vacancy.show()


# процент вакансий с указанной зарплатой
total_vacancies = df_vacancy_total['count'][0]
vacancies_not_null = df_vacancy_not_null['count'][0]
vacancies_null = total_vacancies - vacancies_not_null

null_percent = vacancies_null / total_vacancies * 100
not_null_percent = vacancies_not_null / total_vacancies * 100

data = [null_percent, not_null_percent]

pie_salary = px.pie(
    data,
    values = data,
    names = ['Одна из границ ЗП не указана', 'Указана хотя бы одна граница ЗП'],
    title = "Процент вакансий с указанием ЗП",
    color_discrete_sequence = ['red', 'green']
)

pie_salary.update_traces(pull = [0.1, 0])

pie_salary.write_image("images/salary_specified.png")

pie_salary.show();


# требования по опыту работы
pie_experience = px.pie(
    df_experience,
    values = 'cnt',
    names = 'experience',
    title = "Требования к наличию опыта работы",
    color_discrete_sequence = ['orange', 'yellow', 'green', 'blue']
)


pie_experience.write_image("images/experience.png")

pie_experience.show();


# взаимосвязь между рабочим графиком, типом занятости и количеством вакансий
scatter_schedule = px.scatter(
    df_schedule,
    x = 'employment',
    y = 'schedule',
    color = 'cnt',
    size = 'cnt',
    size_max = 50,
    labels = {'schedule':'График работы', 'employment':'Тип занятости', 'cnt':'Количество вакансий'},
    title = "Взаимосвязь между рабочим графиком и количеством вакансий",
)

scatter_schedule.write_image("images/schedule_employment.png")

scatter_schedule.show()



# Юнит 5. Анализ работодателей

1. Какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [321]:
# текст запроса
query_5_1 = f'''

SELECT
    employer.name,
    COUNT(vacancy.id) AS cnt
FROM
    public.vacancies AS vacancy
JOIN
    public.employers AS employer ON vacancy.employer_id = employer.id
GROUP BY
    employer.id
ORDER BY cnt DESC
LIMIT 5

'''

In [322]:
# результат запроса
df_employers = pd.read_sql_query(query_5_1, connection)

print('Топ пять работодателей по количеству вакансий:')
display(df_employers)

Топ пять работодателей по количеству вакансий:


,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Количество работодателей для каждого региона и вакансий в нём.
Среди регионов, в которых нет вакансий, найдём тот, в котором наибольшее количество работодателей.


In [323]:
# текст запроса
query_5_2 = f'''

SELECT
    area.name                AS area_name,
    COUNT(vacancy.id)        AS vacancy_count,
    COUNT(employer.id)       AS employer_count
FROM
    public.areas area
LEFT JOIN
    public.vacancies vacancy ON area.id = vacancy.area_id
LEFT JOIN
    public.employers employer ON area.id = employer.area
GROUP BY
    area.id
ORDER BY
    vacancy_count ASC, employer_count DESC

'''

In [324]:
# результат запроса

# регион без вакансий и с наибольшим количеством работодателей - первый по списку

print('Регионы по количеству вакансий и работодателей:')
df_regions = pd.read_sql_query(query_5_2, connection)
display(df_regions)

Регионы по количеству вакансий и работодателей:


,area_name,vacancy_count,employer_count
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
1357,Новосибирск,1149438,1149438
1358,Алматы,1364132,1364132
1359,Минск,2354880,2354880
1360,Санкт-Петербург,6320667,6320667


3. Количество регионов с опубликованными вакансиями для каждого работодателя.


In [325]:
# текст запроса
query_5_3 = f'''

SELECT
    employer.name                        AS employer_name,
    COUNT(DISTINCT vacancy.area_id)      AS area_count
FROM
    public.vacancies vacancy
JOIN
    public.employers employer ON vacancy.employer_id = employer.id
GROUP BY
    employer.id
ORDER BY
    area_count DESC

'''

In [326]:
# результат запроса
print('Количество регионов с опубликованными вакансиями для каждого работодателя:')
df_regions_vacancies = pd.read_sql_query(query_5_3, connection)
display(df_regions_vacancies)

Количество регионов с опубликованными вакансиями для каждого работодателя:


,employer_name,area_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Количество работодателей, у которых не указана сфера деятельности.

In [327]:
# текст запроса
query_5_4 = f'''

SELECT
    COUNT(*)
FROM 
    public.employers employer
LEFT JOIN
    public.employers_industries employer_industry ON employer.id = employer_industry.employer_id
WHERE
    employer_industry.employer_id IS NULL

'''

In [328]:
# результат запроса
print('Количество работодателей, у которых не указана сфера деятельности:')
df_null_industry = pd.read_sql_query(query_5_4, connection)
display(df_null_industry)

Количество работодателей, у которых не указана сфера деятельности:


,count
0,8419


5. Узнаем название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [329]:
# текст запроса
query_5_5 = f'''

SELECT
    employer.name
FROM
    public.employers_industries employer_industry
JOIN
    public.employers employer ON employer_industry.employer_id = employer.id
GROUP BY
    employer.id
HAVING 
    COUNT(industry_id) = 4
ORDER BY
    employer.name
OFFSET 2
LIMIT 1

'''

In [330]:
# результат запроса
print('Третья по алфавиту компания, у которой указаны 4 сферы деятельности:')
df_four_industry = pd.read_sql_query(query_5_5, connection)
display(df_four_industry)

Третья по алфавиту компания, у которой указаны 4 сферы деятельности:


,name
0,2ГИС


6. У какого количества работодателей в качестве сферы деятельности указана "Разработка программного обеспечения".


In [331]:
# текст запроса
query_5_6 = f'''

SELECT
    COUNT(DISTINCT employer_industry.employer_id)
FROM
    public.employers_industries employer_industry
JOIN
    public.industries industry ON employer_industry.industry_id = industry.id
WHERE
    industry.name = 'Разработка программного обеспечения'

'''

In [332]:
# результат запроса
print('Количество компаний в сфере "Разработка программного обеспечения":')
df_count_software = pd.read_sql_query(query_5_6, connection)
display(df_count_software)

Количество компаний в сфере "Разработка программного обеспечения":


,count
0,3553


7. Список регионов-миллионников, в которых представлены вакансии компании «Яндекс» вместе с количеством вакансий в этих регионах.

In [333]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

cities_data = soup.select('table.standard.sortable > tbody > tr > td > a')
million_cities = [x.text for x in cities_data]

city_query = ', '.join([f'\'{x}\'' for x in million_cities])

print('Список городов-миллионников:')
print(city_query)

Список городов-миллионников:
'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград'


In [334]:
# текст запроса
query_5_7 = f'''

WITH vacancies_by_region AS
(
    SELECT
        area.name,
        COUNT(vacancy.id) AS cnt
    FROM
        public.vacancies vacancy
    JOIN
        public.employers employer ON vacancy.employer_id = employer.id
    JOIN
        public.areas area ON vacancy.area_id = area.id
    WHERE
        employer.name = 'Яндекс'
        AND
        area.name IN ({city_query})
    GROUP BY
        area.id
    ORDER BY
        cnt
)


SELECT
    *
FROM
    vacancies_by_region

UNION ALL

SELECT
    'Total',
    SUM(cnt)
FROM
    vacancies_by_region

'''

In [335]:
# результат запроса
print('Города-миллионники с вакансиями от компании "Яндекс":')
df_yandex_mil = pd.read_sql_query(query_5_7, connection)
display(df_yandex_mil)

Города-миллионники с вакансиями от компании "Яндекс":


,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Ростов-на-Дону,25.0
5,Казань,25.0
6,Пермь,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


# Выводы по анализу работодателей

В пятерку лидеров по количеству вакансий входят государственные (или с большой долей государственного участия) компании.

Наибольшее количество открытых вакансий размещает компания "Яндекс".

Также "Яндекс" размещает вакансии в наибольшем числе регионов.

В качестве регионов в базе представлены как страны так и города этих стран. Работодатели не размещают вакансии по странам вцелом, предпочитая указывать конкретный город.

Треть работодателей не указывают сферу деятельности.

15% работодателей занимаются разработкой программного обеспечения.

Компания "Яндекс" предоставляет вакансии во всех городах-миллионниках.


<div>
    <img src="images/vacancy_by_employer.png">
</div>
<br/>
<div>
    <img src="images/industry_specified.png">
</div>
<br/>
<div>
    <img src="images/software_dev.png">
</div>
<br/>
<div>
    <img src="images/yandex_mil_vacancy.png">
</div>

***

In [336]:
# построение графиков по анализу работодателей

# работодатели с наибольшим количеством вакансий
bar_employer_vacancy = px.bar(
    df_employers.head(5),
    x = 'name',
    y = 'cnt',
    color = 'name',
    color_discrete_sequence = ['orange', 'yellow', 'green', 'blue', 'teal'],
    labels = { 'cnt':'Количество вакансий', 'area':'Город' },
    title = 'Топ пять работодателей по количеству вакансий'
)

bar_employer_vacancy.update_layout(
    title_x = 0.5
)

bar_employer_vacancy.write_image("images/vacancy_by_employer.png")

bar_employer_vacancy.show()


# процент работодателей с указанной сферой деятельности
total_employers = df_employer_total['count'][0]
industry_null = df_null_industry['count'][0]
industry_not_null = total_employers - industry_null


null_percent = industry_null / total_employers * 100
not_null_percent = industry_not_null / total_employers * 100

data = [null_percent, not_null_percent]

pie_employer_industry = px.pie(
    data,
    values = data,
    names = ['Не указана', 'Указана'],
    title = "Процент работодателей с указанной сферой деятельности",
    color_discrete_sequence = ['green', 'red']
)

pie_employer_industry.update_traces(pull = [0.1, 0])

pie_employer_industry.write_image("images/industry_specified.png")

pie_employer_industry.show();


# процент работодателей, занимающихся разработкой ПО
software_developers = df_count_software['count'][0]
everyone_else = total_employers - software_developers

dev_percent = software_developers / total_employers * 100
other_percent = everyone_else / total_employers * 100

data = [dev_percent, other_percent]

pie_software_dev = px.pie(
    data,
    values = data,
    names = ['Разработка ПО', 'Остальные'],
    title = "Процент работодателей в сфере разработки ПО",
    color_discrete_sequence = ['yellow', 'orange']
)

pie_software_dev.update_traces(pull = [0.1, 0])

pie_software_dev.write_image("images/software_dev.png")

pie_software_dev.show();


# распределение вакансий от Яндекса по городам-миллионникам
bar_yandex_mil_vacancy = px.bar(
    df_yandex_mil.head(16).sort_values(by = 'cnt', ascending = False),
    x = 'name',
    y = 'cnt',
    color = 'name',
    labels = { 'cnt':'Количество вакансий', 'name':'Город-миллионник' },
    title = 'Распределение вакансий от Яндекса по городам-миллионникам'
)

bar_yandex_mil_vacancy.update_layout(
    title_x = 0.5
)

bar_yandex_mil_vacancy.write_image("images/yandex_mil_vacancy.png")

bar_yandex_mil_vacancy.show()


# Юнит 6. Предметный анализ

1. Количество вакансий, имеющих отношение к данным (в названии содержатся слова 'data' или 'данн')

In [337]:
# текст запроса
query_6_1 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies
WHERE
    name ILIKE '%data%'
    OR
    name ILIKE '%данн%'

'''

In [338]:
# результат запроса
print('Количество вакансий, имеющих отношение к данным:')
df_data_vacancy = pd.read_sql_query(query_6_1, connection)
display(df_data_vacancy)

Количество вакансий, имеющих отношение к данным:


,count
0,1771


2. Количество вакансий для начинающего дата-сайентиста.
Вакансиями для дата-сайентистов считаем такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.

In [339]:
# текст запроса

# с таким условием выборку нам попортят значения типа AML или QML, но по условию игнорируем только HTML

query_6_2 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies vacancy
WHERE
    (
        vacancy.name ~ '^(?=.*ML)(?!.*HTML).*$'
        OR
        vacancy.name ~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'
    )
    AND
    (
        vacancy.name ~* 'junior'
        OR
        vacancy.experience = 'Нет опыта'
        OR
        vacancy.employment = 'Стажировка'
    )

'''

# общее количество DS вакансий без учёта тайтла и опыта - пригодится для выводов
query_6_2_1 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies vacancy
WHERE
    (
        vacancy.name ~ '^(?=.*ML)(?!.*HTML).*$'
        OR
        vacancy.name ~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'
    )

'''

In [340]:
# результат запроса
print('Количество вакансий для Junior Data Scientist:')
df_junior_ds_vacancy = pd.read_sql_query(query_6_2, connection)
display(df_junior_ds_vacancy)

df_ds_vacancy = pd.read_sql_query(query_6_2_1, connection)

Количество вакансий для Junior Data Scientist:


,count
0,51


3. Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres

In [341]:
# текст запроса
query_6_3 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies vacancy
WHERE
    vacancy.key_skills ~* '(SQL|postgres)'
    AND
    (
        vacancy.name ~ '^(?=.*ML)(?!.*HTML).*$'
        OR
        vacancy.name ~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'
    )

'''

In [342]:
# результат запроса
print('Количество вакансий для DS с навыками SQL или PostgreSQL:')
df_sql_postgres_skills = pd.read_sql_query(query_6_3, connection)
display(df_sql_postgres_skills)

Количество вакансий для DS с навыками SQL или PostgreSQL:


,count
0,201


4. Количество вакансий для DS, в которых в качестве ключевого навыка указан Python.

In [343]:
# текст запроса
query_6_4 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies vacancy
WHERE
    vacancy.key_skills ~* '(python)'
    AND
    (
        vacancy.name ~ '^(?=.*ML)(?!.*HTML).*$'
        OR
        vacancy.name ~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'
    )

'''

In [344]:
# результат запроса
print('Количество вакансий для DS c навыком Python:')
df_python_skills = pd.read_sql_query(query_6_4, connection)
display(df_python_skills)

Количество вакансий для DS c навыком Python:


,count
0,351


5. Среднее количество ключевых навыков в вакансиях для DS

In [345]:
# текст запроса
query_6_5 = f'''

SELECT
      ROUND( AVG( 1 + LENGTH(vacancy.key_skills) - LENGTH( REPLACE(vacancy.key_skills, CHR(9), '') ) ), 2 )
FROM
    public.vacancies vacancy
WHERE
    vacancy.name ~ '^(?=.*ML)(?!.*HTML).*$'
    OR
    vacancy.name ~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'

'''

In [346]:
# результат запроса
print('Среднее количество ключевых навыков в вакансиях для DS:')
df_ds_skills_avg = pd.read_sql_query(query_6_5, connection)
display(df_ds_skills_avg)

Среднее количество ключевых навыков в вакансиях для DS:


,round
0,6.41


6. На какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет.

Условия:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.

In [347]:
# текст запроса
query_6_6 = f'''

SELECT
    ROUND(AVG(salary.case))
FROM
(
    SELECT
        CASE
            WHEN vacancy.salary_from IS NULL
            THEN vacancy.salary_to
            WHEN vacancy.salary_to IS NULL
            THEN vacancy.salary_from
            ELSE ( vacancy.salary_from + vacancy.salary_to ) / 2
        END
    FROM
        public.vacancies vacancy
    WHERE
        vacancy.experience = 'От 3 до 6 лет'
        AND
        (
            vacancy.salary_from IS NOT NULL
            OR
            vacancy.salary_to IS NOT NULL
        )
        AND
        (
            vacancy.name ~ '^(?=.*ML)(?!.*HTML).*$'
            OR
            vacancy.name ~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'
        )
) salary

'''

In [348]:
# результат запроса
print('Средняя зарплата DS с опытом работы от 3 до 6 лет:')
df_ds_salary_avg = pd.read_sql_query(query_6_6, connection)
display(df_ds_salary_avg)

Средняя зарплата DS с опытом работы от 3 до 6 лет:


,round
0,243115.0


***

# Выводы по предметному анализу

Всего 3.6% вакансий связаны с работой с данными.

При этом отношение вакансий, подходящих по параметрам для Junior Data Scientist к остальным вакансиям - менее одного процента. А когда я был молод для входа в айти достаточно было иметь связную речь, две рабочие руки и способность дорисовать ёжику яблочки на картинке с ёжиком. Кстати, некоторые из этих кандидатов уже успешные руководители.

Но шутки в стороны: на самом деле очевидно, что айти в стране востребовано, но не связанных с айти специальностей несоизмеримо больше, как и вакансий под эти специальности.

Соискателям без опыта или на роль Junior DS подойдут 10.6% вакансий так или иначе связанных с Data Science.

DS вакансии требуют владения в среднем 7 ключевыми навыками.

Соискатели с опытом работы от 3 до 6 лет могут претендовать на среднюю зарплату равную 243 тысячам, что в принципе справедливо не только для DS и является вполне адекватной рыночной зарплатой среднестатистического айти инженера с таким опытом.

<div>
    <img src="images/data_vacancies.png">
</div>
<br/>

<div>
    <img src="images/data_vacancies_junior_ds.png">
</div>
<br/>

<div>
    <img src="images/data_vacancies_junior_ds_ds.png">
</div>
<br/>

In [349]:
# построение графиков по предметному анализу

# процент вакансий, имеющих отношение к данным
total_vacancies = df_vacancy_total['count'][0]
vacancies_data = df_data_vacancy['count'][0]
vacancies_other = total_vacancies - vacancies_data

data_percent = vacancies_data / total_vacancies * 100
other_percent = vacancies_other / total_vacancies * 100

data = [data_percent, other_percent]

pie_data_vacancy = px.pie(
    data,
    values = data,
    names = ['Работа с данными', 'Прочее'],
    title = "Процент вакансий, имеющих отношение к работе с данными",
    color_discrete_sequence = ['yellow', 'orange']
)

pie_data_vacancy.update_traces(pull = [0.1, 0])

pie_data_vacancy.write_image("images/data_vacancies.png")

pie_data_vacancy.show();


# деморализующий результат для junior data scientist
total_vacancies = df_vacancy_total['count'][0]
junior_data = df_junior_ds_vacancy['count'][0]


junior_percent = junior_data / total_vacancies * 100

data = [junior_percent, other_percent]

pie_data_junior = px.pie(
    data,
    values = data,
    names = ['Junior DS', 'Все вакансии'],
    title = "Мои соболезнования джунам",
    color_discrete_sequence = ['yellow', 'orange']
)

pie_data_junior.update_traces(pull = [0.1, 0])

pie_data_junior.write_image("images/data_vacancies_junior_ds.png")

pie_data_junior.show();


# отношение подходящих для джуна вакансий ко всем DS вакансиям
total_ds_vacancies = df_ds_vacancy['count'][0]
junior_data = df_junior_ds_vacancy['count'][0]
vacancies_other = total_ds_vacancies - junior_data

junior_percent = junior_data / total_ds_vacancies * 100
other_percent = vacancies_other / total_ds_vacancies * 100

data = [junior_percent, other_percent]

pie_data_junior_ds = px.pie(
    data,
    values = data,
    names = ['Junior DS', 'DS Прочее'],
    title = "Отношение подходящих для джуна вакансий ко всем DS вакансиям",
    color_discrete_sequence = ['yellow', 'orange']
)

pie_data_junior_ds.update_traces(pull = [0.1, 0])

pie_data_junior_ds.write_image("images/data_vacancies_junior_ds_ds.png")

pie_data_junior_ds.show();


# Дополнительные исследования

1. Зависимость средней зарплаты от опыта работы по всем вакансиям, за исключением DS

In [350]:
# текст запроса
query_salary_exp = f'''

SELECT
    salary.experience,
    ROUND(AVG(salary.case))
FROM
(
    SELECT
        vacancy.experience,
        CASE
            WHEN vacancy.salary_from IS NULL
            THEN vacancy.salary_to
            WHEN vacancy.salary_to IS NULL
            THEN vacancy.salary_from
            ELSE ( COALESCE(vacancy.salary_from, 0) + COALESCE(vacancy.salary_to,0) ) / 2 
        END
    FROM
        public.vacancies vacancy
    WHERE
        vacancy.name !~ '^(?=.*ML)(?!.*HTML).*$'
        AND
        vacancy.name !~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'
) salary
GROUP BY
    salary.experience

'''

In [351]:
# результат запроса
print('Зависимость средней зарплаты от опыта работы:')
df_salary_exp = pd.read_sql_query(query_salary_exp, connection)
display(df_salary_exp)

Зависимость средней зарплаты от опыта работы:


,experience,round
0,Более 6 лет,166256.0
1,От 3 до 6 лет,132683.0
2,Нет опыта,40425.0
3,От 1 года до 3 лет,76403.0


2. Зависимость средней зарплаты от опыта работы по вакансиям DS

In [352]:
# текст запроса
query_salary_exp_ds = f'''

SELECT
    salary.experience,
    ROUND(AVG(salary.case))
FROM
(
    SELECT
        vacancy.experience,
        CASE
            WHEN vacancy.salary_from IS NULL
            THEN vacancy.salary_to
            WHEN vacancy.salary_to IS NULL
            THEN vacancy.salary_from
            ELSE ( COALESCE(vacancy.salary_from, 0) + COALESCE(vacancy.salary_to,0) ) / 2 
        END
    FROM
        public.vacancies vacancy
    WHERE
        vacancy.name ~ '^(?=.*ML)(?!.*HTML).*$'
        OR
        vacancy.name ~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'

) salary
GROUP BY
    salary.experience

'''

In [353]:
# результат запроса
print('Зависимость средней зарплаты от опыта работы по вакансиям DS:')
df_salary_exp_ds = pd.read_sql_query(query_salary_exp_ds, connection)
display(df_salary_exp_ds)

Зависимость средней зарплаты от опыта работы по вакансиям DS:


,experience,round
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


3. Посмотрим на топ 20 самых востребованных навыков в DS

In [354]:
# текст запроса
query_top_skills_ds = f'''

SELECT
    skills.skill_name,
    COUNT(skills.skill_name) AS cnt
FROM
(
    SELECT
        SPLIT_PART(vacancy.key_skills, CHR(9), 2) AS skill_name
    FROM
        public.vacancies vacancy
    WHERE
        vacancy.name ~ '^(?=.*ML)(?!.*HTML).*$'
        OR
        vacancy.name ~* '(data scientist|data science|исследователь данных|machine learning|.*машинн.*обучен.*)'
) skills
WHERE
    skills.skill_name != ''
GROUP BY
    skills.skill_name
ORDER BY cnt DESC
LIMIT 20

'''

In [355]:
# результат запроса
print('Топ 20 самых востребованных навыков в DS:')
df_top_skills_ds = pd.read_sql_query(query_top_skills_ds, connection)
display(df_top_skills_ds)

Топ 20 самых востребованных навыков в DS:


,skill_name,cnt
0,SQL,100
1,Machine Learning,24
2,Python,24
3,PyTorch,18
4,Git,17
5,ООП,16
6,Linux,14
7,MS PowerPoint,14
8,machine learning,12
9,Spark,12


# Выводы по дополнительным исследованиям

Средняя зарплата ожидаемо растёт с ростом опыта работы. В сегменте DS зарплата без опыта работы соизмерима со средней зарплатой с опытом от 1 до 3 лет по другим вакансиям.

Примечательно, что отсутствуют данные по зарплате соискателей в сфере DS с опытом работы более 6 лет. Видимо в массе своей они уже не ищут вакансии на HH, либо зарплата там договорная и сильно варьируется.

Самым востребованным навыком является SQL.

<div>
    <img src="images/salary_exp.png">
</div>
<br/>

<div>
    <img src="images/salary_exp_ds.png">
</div>
<br/>

<div>
    <img src="images/skills_ds.png">
</div>
<br/>



In [356]:
# построение графиков по дополнительным исследованиям

# Зависимость средней зарплаты от опыта работы:
bar_salary_exp = px.bar(
    df_salary_exp.sort_values(by='experience'),
    x = 'experience',
    y = 'round',
    color = 'experience',
    color_discrete_sequence = ['orange', 'yellow', 'green', 'blue', 'teal'],
    labels = { 'round':'Средняя зарплата', 'experience':'Опыт работы' },
    title = 'Зависимость средней зарплаты от опыта работы'
)

bar_salary_exp.update_layout(
    title_x = 0.5
)

bar_salary_exp.write_image("images/salary_exp.png")

bar_salary_exp.show()


# Зависимость средней зарплаты в DS от опыта работы:
bar_salary_exp_ds = px.bar(
    df_salary_exp_ds.sort_values(by='experience'),
    x = 'experience',
    y = 'round',
    color = 'experience',
    color_discrete_sequence = ['orange', 'yellow', 'green', 'blue', 'teal'],
    labels = { 'round':'Средняя зарплата', 'experience':'Опыт работы' },
    title = 'Зависимость средней зарплаты в DS от опыта работы'
)

bar_salary_exp_ds.update_layout(
    title_x = 0.5
)

bar_salary_exp_ds.write_image("images/salary_exp_ds.png")

bar_salary_exp_ds.show()


# топ 20 востребованных DS навыков по частоте упоминания в вакансиях
scatter_skills = px.scatter(
    df_top_skills_ds,
    y = 'skill_name',
    x = 'cnt',
    color = 'cnt',
    size = 'cnt',
    size_max = 50,
    labels = {'skill_name':'Навык', 'cnt':'Частота упоминания'},
    title = "Частота упоминания навыков в вакансиях DS",
)

scatter_skills.write_image("images/skills_ds.png")

scatter_skills.show()

# Общий вывод по проекту

Большинство вакансий сосредоточены в таких городах, как Москва, Санкт-Петербург, Минск, Новосибирск и Алматы. Из этой пятёрки Москва лидирует с большим отрывом.

Свыше половины вакансий требуют наличие опыта работы от 1 года до 3 лет. Наиболее востребованы сотрудники на полную занятость и полный рабочий день.

Основными работодателями являются крупные государственные компании. Лидером по количеству вакансий является компания "Яндекс".

Средние показатели заработной платы лежат в диапазоне от 70 до 110 тысяч. У половины вакансий не указана заработная плата, что сказывается на достоверности вычисления среднего значения.

Джуниорам в наше время приходится нелегко.

Данные по ключевым навыкам можно нормализовать - вынести в отдельную таблицу.

Таблица регионов содержит как города, так и страны. Страны, судя по всему, особого интереса не представляют. Можно нормализовать таблицу добавив тип региона или избавиться от стран.

In [357]:
connection.close()